In [418]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import json

mongoimport --type csv -d project3 -c emissions --headerline --drop GCB2022v27_MtCO2_flat.csv

In [419]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [420]:
mongo.list_database_names()

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'fruits_db',
 'local',
 'met',
 'petsitly_marketing',
 'project3',
 'uk_food']

In [421]:
db = mongo["project3"]

In [422]:
db.list_collection_names()

['emissions']

In [423]:
emissions = db["emissions"]
emissions

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'project3'), 'emissions')

In [424]:
pprint(emissions.find_one())

{'Cement': '',
 'Coal': '',
 'Country': 'Afghanistan',
 'Flaring': '',
 'Gas': '',
 'ISO 3166-1 alpha-3': 'AFG',
 'Oil': '',
 'Other': '',
 'Per Capita': '',
 'Total': 0,
 'Year': 1750,
 '_id': ObjectId('66523c4896b122622ff55ef4')}


In [425]:
query = {"Year" : {"$gte" : 1922}}

In [426]:
emissions_df = pd.DataFrame(emissions.find(query))
emissions_df.head()

,_id,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,66523c4896b122622ff55fa0,Afghanistan,AFG,1922,0,,,,,,,
1,66523c4896b122622ff55fa1,Afghanistan,AFG,1923,0,,,,,,,
2,66523c4896b122622ff55fa2,Afghanistan,AFG,1924,0,,,,,,,
3,66523c4896b122622ff55fa3,Afghanistan,AFG,1925,0,,,,,,,
4,66523c4896b122622ff55fa4,Afghanistan,AFG,1927,0,,,,,,,


In [427]:
emissions_df.head()

,_id,Country,ISO 3166-1 alpha-3,Year,Total,Coal,Oil,Gas,Cement,Flaring,Other,Per Capita
0,66523c4896b122622ff55fa0,Afghanistan,AFG,1922,0,,,,,,,
1,66523c4896b122622ff55fa1,Afghanistan,AFG,1923,0,,,,,,,
2,66523c4896b122622ff55fa2,Afghanistan,AFG,1924,0,,,,,,,
3,66523c4896b122622ff55fa3,Afghanistan,AFG,1925,0,,,,,,,
4,66523c4896b122622ff55fa4,Afghanistan,AFG,1927,0,,,,,,,


In [428]:
del emissions_df["_id"]
del emissions_df["ISO 3166-1 alpha-3"]

In [429]:
emissions_json = emissions_df.to_json(orient="records")

In [430]:
parsed = json.loads(emissions_json)
parsed

[{'Country': 'Afghanistan',
  'Year': 1922,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''},
 {'Country': 'Afghanistan',
  'Year': 1923,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''},
 {'Country': 'Afghanistan',
  'Year': 1924,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''},
 {'Country': 'Afghanistan',
  'Year': 1925,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''},
 {'Country': 'Afghanistan',
  'Year': 1927,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''},
 {'Country': 'Afghanistan',
  'Year': 1928,
  'Total': 0,
  'Coal': '',
  'Oil': '',
  'Gas': '',
  'Cement': '',
  'Flaring': '',
  'Other': '',
  'Per Capita': ''}

In [431]:
#https://stackoverflow.com/questions/60029873/pandas-to-json-redundant-backslashes
with open("data.json", "w", encoding="utf8") as output:
    output.write(json.dumps(parsed, indent=4))